In [1]:
import mindspore
from mindspore import nn, ops, Tensor
from mindspore.train import Model
import mindspore.numpy as mnp
import numpy as np
from mindspore.common.initializer import XavierUniform
from mindspore import context
context.set_context(mode=1, device_target="CPU")
from src.model import HeteroNet

In [2]:
model = HeteroNet(embed_size=32, clu_node_dim=6, clu_edge_dim=9, trk_node_dim=5, trk_edge_dim=5)

In [3]:
import dgl
graphs = dgl.load_graphs('data/sig/sig_2jvv_m50GeV_t1_1_100_heter.bin')
g = graphs[0]
l = graphs[1]['labels'].numpy()

In [4]:
import mindspore.dataset as mds
ds = mds.NumpySlicesDataset((l, g), column_names=['label', 'graph'], sampler=mds.RandomSampler())

In [5]:
        clu_ndata = Tensor(g[0].nodes['cluster'].data['scalars'].numpy(), mindspore.float32)
        print(clu_ndata.shape)
        clu_edata = Tensor(g[0].edges['clu_inter'].data['interact'].numpy(), mindspore.float32)
        print(clu_edata.shape)
        clu_src, clu_dst = g[0].edges(etype='clu_inter')
        clu_src = Tensor(clu_src.numpy())
        clu_dst = Tensor(clu_dst.numpy())
        print(clu_src.shape)
        trk_ndata = Tensor(g[0].nodes['tracker'].data['scalars'].numpy(), mindspore.float32)
        print(trk_ndata.shape)
        trk_edata = Tensor(g[0].edges['trk_inter'].data['interact'].numpy(), mindspore.float32)
        print(trk_edata.shape)
        trk_src, trk_dst = g[0].edges(etype='trk_inter')
        trk_src = Tensor(trk_src.numpy())
        trk_dst = Tensor(trk_dst.numpy())
        print(trk_src.shape, type(trk_src[0]), trk_src[0])

(4, 6)
(12, 9)
(12,)
(10, 5)
(90, 5)
(90,) <class 'mindspore.common._stub_tensor.StubTensor'> 0


In [6]:
model(clu_ndata, clu_edata, clu_src, clu_dst, trk_ndata, trk_edata, trk_src, trk_dst)

(4, 32) (12, 32) (10, 32) (90, 32)
(12, 32) (12, 32) (12, 32)
(12, 96)
(4, 32) (12, 32)
(90, 32) (90, 32) (90, 32)
(90, 96)
(10, 32) (90, 32)
(12, 32) (12, 32) (12, 32)
(12, 96)
(4, 32) (12, 32)
(90, 32) (90, 32) (12, 32)


RuntimeError: For 'Gather', the 'input_indices' should be in the range [0, 4), but got 4, error_code[58]

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ops/kernel/cpu/gather_cpu_kernel.cc:113 operator()
